In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN, OPTICS, KMeans
import matplotlib.pyplot as plt
import seaborn as sns
#from module import *
%matplotlib inline
def raman_plot():
    """
    Funzione che serve per settare la dimensione dell'immgine e i label degli assi per un plot di spettro Raman.
    """
    plt.figure(figsize=(16,6))
    plt.xlabel("wave number [1/cm]")
    plt.ylabel("Intensity")

In [6]:
# import dei centroidi
data = pd.read_csv("../data/processed/CLUSTERING_data_centres.csv")

In [9]:
# import degli spettri puri
# definisco i nomi dei vari materiali usando il file che li contiene tutti
pure_material_names=[]
with open('../data/raw/Database Raman/BANK_LIST.dat') as f:
    pure_material_names=[i[1:len(i)-5] for i in f.readlines()]
l=[]
# calcolo la dimensione del materiale puro con più dati
for i in range(len(pure_material_names)):
    l.append(pd.read_csv('../data/raw/Database Raman/'+pure_material_names[i]+'.txt', delim_whitespace=True, names=[pure_material_names[i]+'_wl',pure_material_names[i]+'_I']).size)
max_size=int(max(l)/2)
# genero un dataframe vuoto per poter usare il metodo join
pure_materials = pd.DataFrame(np.zeros(max_size),columns=['empty'])
# importiamo i dati: nome_I (intensità) e nome_wn
for i in range(len(pure_material_names)):
    pure_materials=pure_materials.join(pd.read_csv('../data/raw/Database Raman/'+pure_material_names[i]+'.txt', delim_whitespace=True, names=[pure_material_names[i]+'_wn',pure_material_names[i]+'_I']))
    
pure_materials.drop('empty', axis = 1,inplace=True)

Ora devo interpolare gli spettri puri ai dati

In [25]:
pure_materials_interpoled=pd.DataFrame(data.wn.copy())
for temp in pure_material_names:
    pure_materials_interpoled=pure_materials_interpoled.join(pd.DataFrame(np.interp(data.wn, pure_materials[temp+'_wn'] ,pure_materials[temp+'_I']),columns=[temp]))


In [27]:
#Normalizzazione
for i in pure_material_names:
    pure_materials_interpoled[i]=pure_materials_interpoled[i]/np.trapz(abs(pure_materials_interpoled[i].dropna()), x=pure_materials_interpoled[i])

In [29]:
pure_materials_interpoled.head()

,wn,Albite,Almandine,Amesite,Anatase,Andradite,Antigorite,Armalcolite,Arsenopyrite,Augite,...,Pyrophyllite,Quartz,Siderite,Spinel,Szomolnokite,Talc,Titanite,Tremolite,Zircon,Zoisite
0,184.0,-0.000227,-0.015984,-0.002342,-0.007878,-0.004560,-0.008177,0.0,-0.004824,-0.004110,...,-0.035991,-0.004876,-0.005696,-0.785261,-0.009674,-0.003676,-0.001134,-0.022605,-0.007181,-0.001300
1,185.0,-0.000225,-0.011871,-0.002873,-0.006370,-0.004200,-0.008576,0.0,-0.005228,-0.004117,...,-0.035558,-0.005350,-0.005877,-1.286136,-0.011507,-0.004178,-0.001139,-0.015591,-0.008345,-0.001415
2,186.0,-0.000215,-0.010829,-0.003469,-0.004698,-0.003901,-0.007864,0.0,-0.005562,-0.003877,...,-0.056632,-0.005895,-0.006129,-1.472472,-0.012420,-0.004868,-0.001089,-0.014400,-0.009806,-0.001524
3,187.0,-0.000199,-0.008271,-0.004154,-0.002379,-0.003429,-0.007897,0.0,-0.006019,-0.003543,...,-0.086351,-0.006466,-0.006412,-1.031640,-0.011859,-0.006109,-0.000955,-0.009475,-0.010459,-0.001647
4,188.0,-0.000180,-0.005527,-0.004877,-0.000414,-0.003007,-0.008976,0.0,-0.005964,-0.003484,...,-0.116858,-0.007050,-0.006618,-0.587861,-0.012285,-0.007753,-0.000767,-0.010622,-0.011117,-0.001753
